In [1]:
import os
import time
import datetime
import pyspark.sql.functions as sf
# from uuid import *
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import when
from pyspark.sql.functions import col
# from pyspark.sql.types import *
from pyspark.sql.functions import lit
from pyspark import SparkConf, SparkContext
from uuid import UUID
import time_uuid
from pyspark.sql import Row
from pyspark.sql.functions import udf
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.window import Window as W

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

packages = ["mysql:mysql-connector-java:8.0.30",
"com.datastax.spark:spark-cassandra-connector_2.12:3.1.0"]
packages_str = ','.join(packages)
# .config("spark.jars.packages", packages_str) \
# khởi tạo session
spark = SparkSession.builder \
.config("spark.jars.packages", packages_str) \
.config("spark.cassandra.connection.host", "localhost")\
.config("spark.cassandra.connection.port", "9042").getOrCreate()

    # Retrieve the SparkConf object from the SparkContext
#conf = spark.sparkContext.getConf()
#conf.getAll()

your 131072x1 screen size is bogus. expect trouble
23/08/23 02:27:33 WARN Utils: Your hostname, tester resolves to a loopback address: 127.0.1.1; using 172.25.162.57 instead (on interface eth0)
23/08/23 02:27:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/yourname/miniconda3/envs/sparky/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
mysql#mysql-connector-java added as a dependency
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-37b9b9aa-eb01-4cc4-94b1-81a9c3238354;1.0
	confs: [default]
	found mysql#mysql-connector-java;8.0.30 in central
	found com.google.protobuf#protobuf-java;3.19.4 in central
	found com.datastax.spark#spark-cassandra-connector_2.12;3.1.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.1.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.12.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;

In [3]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

def extract_time(uuid): # convert uuid to datetime 
  my_uuid =  UUID(uuid)
  return time_uuid.TimeUUID(bytes=my_uuid.bytes).get_datetime().strftime("%Y-%m-%d %H:%M:%S") #2022-07-26 06:45:09

def process_df(df): # convert uuuid từ cột creatime thay thế cho cột ts
# Spilt data


  # Cast data String to Int
  df = df.withColumns({'job_id': col('job_id').cast('int'),
                      'bid': col('bid').cast('int'),
                      'group_id': col('group_id').cast('int'),
                      'publisher_id': col('publisher_id').cast('int'),
                      'campaign_id': col('campaign_id').cast('int'),})
  # Filter null data
  df = df.filter(df.job_id.isNotNull()) # nên sử dụng cách này, có tác dụng khi job_id là Int
  # df = df.filter(col('job_id') !='null') # khắc phục tạm vì do lúc lưu data gặp lỗi nên job_id là String

  # Converting function to UDF
  # StringType() is by default hence not required
  extract_timeUDF = udf(lambda x: extract_time(x), StringType())

  df = df.withColumn('ts', extract_timeUDF(col('create_time')))
  
  df = df.select('ts','job_id','custom_track','bid','campaign_id','group_id','publisher_id')

  return df
  # Cách 2: Thay vì dùng UDF có thể collect() cột create-time ra rồi duyệt vòng for đưa vào 1 list mới
  # Cách 3: sử dụng apply(lambda:........)


In [4]:
def calculating_clicks(df):
    clicks_data = df.filter(col('custom_track') == "click")
    clicks_data = clicks_data.na.fill({'bid':0})
    clicks_data = clicks_data.na.fill({'campaign_id':0})
    clicks_data = clicks_data.na.fill({'group_id':0})
    clicks_data = clicks_data.na.fill({'publisher_id':0})
    clicks_data.registerTempTable('clicks') # Tạo bảng tạm để thực hiện truy vấn spark sql
    clicks_output = spark.sql(""" SELECT  job_id, date(ts) AS dates, hour(ts) AS hours, publisher_id, campaign_id, group_id,
                      avg(bid) AS bid_set, count(*) AS clicks, sum(bid) AS spend_hour
        FROM clicks
        GROUP BY job_id, date(ts) , hour(ts) , publisher_id, campaign_id, group_id """)
    return clicks_output # return a SparkDataFrame


In [5]:
def calculating_qualified(df):
    qualified_data = df.filter(col('custom_track') == "qualified")
    qualified_data = qualified_data.na.fill({'bid':0})
    qualified_data = qualified_data.na.fill({'campaign_id':0})
    qualified_data = qualified_data.na.fill({'group_id':0})
    qualified_data = qualified_data.na.fill({'publisher_id':0})
    qualified_data.registerTempTable('qualified') # Tạo bảng tạm để thực hiện truy vấn spark sql
    qualified_output = spark.sql(""" SELECT  job_id, date(ts) AS dates, hour(ts) AS hours, publisher_id, campaign_id, group_id,
                      count(*) AS qualified
        FROM qualified
        GROUP BY job_id, date(ts) , hour(ts) ,publisher_id, campaign_id, group_id """)
    return qualified_output


In [6]:
def process_final_data(clicks_output,qualified_output):
  final_data = clicks_output.join(qualified_output, ['job_id', 'dates', 'hours','publisher_id', 'campaign_id', 'group_id'],'full')
  return final_data


In [7]:
def final_output(df):
    clicks = calculating_clicks(df)
    qualified = calculating_qualified(df)
    return process_final_data(clicks, qualified)

In [8]:
def import_to_sql(output):
    final_output = output.select('job_id','dates','hours','publisher_id','campaign_id','group_id','qualified','clicks','bid_set','spend_hour') # xếp lại thứ tự
    
    final_output.write.mode("append")\
    .format("jdbc")\
    .option("url",  "jdbc:mysql://localhost:3306/ware")\
    .option("dbtable", "events")\
    .option("user",  "root")\
    .option("password","123456")\
    .option("driver",'com.mysql.cj.jdbc.Driver')\
    .save()
    
    return print('Data imported successfully')


In [9]:
def retrieve_mysql_lastest_time(url, driver, user, password):
    sql = """(select max(last_updated_time) from events) A"""
    mysql_time = spark.read.format('jdbc').options(url = url , driver = driver , dbtable = sql, user=user , password = password).load().take(1)[0][0]
    if mysql_time is None:
        time = '1998-01-01 23:59:59'
    else:
        time = mysql_time.strftime("%Y-%m-%d %H:%M:%S")
    return time



In [10]:
def retrieve_cass_lastest_time():
    temp_df = spark.read.format("org.apache.spark.sql.cassandra").options(table="tracking",keyspace="test").load() # read Data
    cass_time = temp_df.agg({'ts':'max'}).take(1)[0][0]

    return cass_time.strftime("%Y-%m-%d %H:%M:%S")

In [11]:
def main(mysql_time):
    
    print("-----------------------------")
    print("Retrieve data from Cassandra and process")
    print("-----------------------------")
    df = spark.read.format("org.apache.spark.sql.cassandra").options(table="tracking",keyspace="test").load().where(col('ts') >= mysql_time) # read Data
    
    print("-----------------------------")
    print("process data")
    print("-----------------------------")
    df = process_df(df)

    print("-----------------------------")
    print("process final output")
    print("-----------------------------")
    cass_output = final_output(df)
    
    print("-----------------------------")
    print("Process data successfully")
    print("-----------------------------")

    import_to_sql(cass_output)
    return 'Task successfully'



In [13]:
url=   "jdbc:mysql://localhost:3306/ware"
driver = "com.mysql.cj.jdbc.Driver"
user = 'root'
password = '123456'

while True:
    timestart = datetime.datetime.now()

    mysql_time = retrieve_mysql_lastest_time(url, driver, user, password)
    print(f'Latest time in MySQL is {mysql_time}')

    cassandra_time = retrieve_cass_lastest_time()
    print(f'Latest time in Cassandra is {cassandra_time}')

    if cassandra_time > mysql_time :
        main(mysql_time)
    else :
        print('No New data found')

    timeend = datetime.datetime.now()
    
    print("Job takes {} seconds to execute".format((timeend - timestart).total_seconds()))

    print("-------------------------------------------")
    time.sleep(30)

Latest time in MySQL is 2023-08-23 00:07:54


23/08/23 02:30:19 WARN V2ScanPartitioningAndOrdering: Spark ignores the partitioning CassandraPartitioning. Please use KeyGroupedPartitioning for better performance


Latest time in Cassandra is 2023-08-23 02:28:46
-----------------------------
Retrieve data from Cassandra and process
-----------------------------
-----------------------------
process data
-----------------------------
-----------------------------
process final output
-----------------------------


/home/yourname/miniconda3/envs/sparky/lib/python3.10/site-packages/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)
23/08/23 02:30:32 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


-----------------------------
Process data successfully
-----------------------------


...

23/08/23 02:32:44 WARN V2ScanPartitioningAndOrdering: Spark ignores the partitioning CassandraPartitioning. Please use KeyGroupedPartitioning for better performance


Latest time in Cassandra is 2023-08-23 02:32:15
-----------------------------
Retrieve data from Cassandra and process
-----------------------------
-----------------------------
process data
-----------------------------
-----------------------------
process final output
-----------------------------
-----------------------------
Process data successfully
-----------------------------


23/08/23 02:32:52 WARN V2ScanPartitioningAndOrdering: Spark ignores the partitioning CassandraPartitioning. Please use KeyGroupedPartitioning for better performance
23/08/23 02:32:52 WARN V2ScanPartitioningAndOrdering: Spark ignores the partitioning CassandraPartitioning. Please use KeyGroupedPartitioning for better performance


Data imported successfully
Job takes 29.131845 seconds to execute
-------------------------------------------


KeyboardInterrupt: 